In [187]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import yfinance as yf
import pandas as pd
import datetime
import time
from arch import arch_model 
import matplotlib.pyplot as plt
from numba import jit
from sklearn.metrics import mean_squared_error as mse 
import plotly.graph_objects as go


#NN
from sklearn.neural_network import MLPRegressor

#TensorFlow - Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')

# 1. Classical Models

## 1.1 Data

In [188]:
stocks = '^GSPC'
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2021, 8, 1)

data = yf.download(stocks, start=start, end= end, interval='1d')

data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-26,4409.580078,4422.729980,4405.450195,4422.299805,4422.299805,4275630000
2021-07-27,4416.379883,4416.379883,4372.509766,4401.459961,4401.459961,4241950000
2021-07-28,4402.950195,4415.470215,4387.009766,4400.640137,4400.640137,4215290000
2021-07-29,4403.589844,4429.970215,4403.589844,4419.149902,4419.149902,4044600000
2021-07-30,4395.120117,4412.250000,4389.649902,4395.259766,4395.259766,3956740000


In [189]:
ret = 100 * (data.pct_change()[1:]['Adj Close'])
realized_vol = ret.rolling(5).std()
retv = ret.values

n = 252
split_date = ret.iloc[-n:].index

In [190]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol, mode='lines', name='Realized Volatility'))
fig.update_layout(
    title=f'Realized Volatility - {stocks}',
    xaxis_title='Date',
    yaxis_title='Volatility',
    width=1000,
    height=600
)
fig.show()

In [191]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[1:], y=ret, mode='lines', name='Realized Volatility'))
fig.update_layout(
    title=f'Volatility Clustering of {stocks}',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## 1.2 ARCH

In [192]:
arch = arch_model(ret, mean='zero', vol='ARCH', p=1).fit(disp='off')
print(arch.summary())

                        Zero Mean - ARCH Model Results                        
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -4061.27
Distribution:                  Normal   AIC:                           8126.54
Method:            Maximum Likelihood   BIC:                           8138.50
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:22   Df Model:                            0
                            Volatility Model                            
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          0.7014  5.002e-02     14.023  1.129e-44 [  0.603,  0.79

In [193]:
bic_arch = []


for p in range(1, 5):
    arch = arch_model(ret, mean='zero', vol='ARCH', p=p).fit(disp='off')
    bic_arch.append(arch.bic)
    if arch.bic == np.min(bic_arch):
        best_param = p
arch = arch_model(ret, mean='zero', vol='ARCH', p=best_param).fit(disp='off')
print(arch.summary)

<bound method ARCHModelResult.summary of                         Zero Mean - ARCH Model Results                        
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -3709.51
Distribution:                  Normal   AIC:                           7429.03
Method:            Maximum Likelihood   BIC:                           7458.91
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:22   Df Model:                            0
                             Volatility Model                             
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2794  

In [194]:
forecast = arch.forecast(start=split_date[0])
forecast_arch = forecast
forecast_arch.variance.index = pd.to_datetime(forecast_arch.variance.index)
realized_vol.index = pd.to_datetime(realized_vol.index)

In [195]:
rmse_arch = np.sqrt(mse(realized_vol[-n:]/100,
                        np.sqrt(forecast_arch.variance.iloc[-len(split_date):]/100
                                )
                        )
                    )

print('The RMSE value of ARCH Model is {:.4f}'.format(rmse_arch))

The RMSE value of ARCH Model is 0.0896


In [196]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[1:], y=(realized_vol / 100), mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=forecast_arch.variance.index, y=(forecast_arch.variance['h.1']/100), mode='lines', name='Predict Volatility'))
fig.update_layout(
    title=f'Volatility Clustering of {stocks} - ARCH',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## 1.3 GARCH

In [197]:
garch = arch_model(ret, mean='zero', vol='GARCH', p=1).fit(disp='off')
garch.summary

<bound method ARCHModelResult.summary of                        Zero Mean - GARCH Model Results                        
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3654.98
Distribution:                  Normal   AIC:                           7315.97
Method:            Maximum Likelihood   BIC:                           7333.90
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:23   Df Model:                            0
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.

In [198]:
bic_garch = []


for p in range(1, 5):
    garch = arch_model(ret, mean='zero', vol='GARCH', p=p).fit(disp='off')
    bic_garch.append(garch.bic)
    if garch.bic == np.min(bic_garch):
        best_param = p
garch = arch_model(ret, mean='zero', vol='GARCH', p=best_param).fit(disp='off')
print(garch.summary)

<bound method ARCHModelResult.summary of                        Zero Mean - GARCH Model Results                        
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3654.98
Distribution:                  Normal   AIC:                           7315.97
Method:            Maximum Likelihood   BIC:                           7333.90
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:23   Df Model:                            0
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.

In [199]:
forecast = garch.forecast(start=split_date[0])
forecast_garch = forecast
forecast_garch.variance.index = pd.to_datetime(forecast_garch.variance.index)
realized_vol.index = pd.to_datetime(realized_vol.index)

In [200]:
rmse_garch = np.sqrt(mse(realized_vol[-n:]/100,
                        np.sqrt(forecast_garch.variance.iloc[-len(split_date):]/100
                                )
                        )
                    )

print('The RMSE value of GARCH Model is {:.4f}'.format(rmse_garch))

The RMSE value of GARCH Model is 0.0878


In [201]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[1:], y=(realized_vol / 100), mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=forecast_garch.variance.index, y=(forecast_garch.variance['h.1']/100), mode='lines', name='Predict Volatility'))
fig.update_layout(
    title=f'Volatility Clustering of {stocks} - GARCH',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## 1.4 GJR - GARCH

In [202]:
bic_gjr_garch = []


for p in range(1, 5):
    for q in range(1, 5):
        gjr_garch = arch_model(ret, mean='zero', p=p, o=1, q=q).fit(disp='off')
        bic_gjr_garch.append(gjr_garch.bic)
        if gjr_garch.bic == np.min(bic_gjr_garch):
            best_param = p, q
garch = arch_model(ret, mean='zero', p=best_param[0], o=1, q=best_param[1]).fit(disp='off')
print(garch.summary)

<bound method ARCHModelResult.summary of                      Zero Mean - GJR-GARCH Model Results                      
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -3591.15
Distribution:                  Normal   AIC:                           7190.30
Method:            Maximum Likelihood   BIC:                           7214.21
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:26   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega         

In [203]:
forecast = gjr_garch.forecast(start=split_date[0])
forecast_garch = forecast
forecast_garch.variance.index = pd.to_datetime(forecast_garch.variance.index)
realized_vol.index = pd.to_datetime(realized_vol.index)

In [204]:
rmse_gjr_garch = np.sqrt(mse(realized_vol[-n:]/100,
                        np.sqrt(forecast_garch.variance.iloc[-len(split_date):]/100
                                )
                        )
                    )

print('The RMSE value of ARCH Model is {:.4f}'.format(rmse_gjr_garch))

The RMSE value of ARCH Model is 0.0880


In [205]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[1:], y=(realized_vol / 100), mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=forecast_garch.variance.index, y=(forecast_garch.variance['h.1']/100), mode='lines', name='Predict Volatility'))
fig.update_layout(
    title=f'Volatility Clustering of {stocks} - GJR-GARCH',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## 1.5 EGARCH

In [206]:
bic_egarch = []

for p in range(1, 5):
    for q in range(1, 5):
        egarch = arch_model(ret, mean='zero', vol='EGARCH', p=p, q=q ).fit(disp='off')
        bic_egarch.append(egarch.bic)
        if egarch.bic == np.min(bic_egarch):
            best_param = p, q
egarch = arch_model(ret, mean='zero', vol='EGARCH', p=best_param[0], q=best_param[1] ).fit(disp='off')
print(egarch.summary)
forecast = egarch.forecast(start=(split_date[0]))
forecast_egarch = forecast

<bound method ARCHModelResult.summary of                        Zero Mean - EGARCH Model Results                       
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                     EGARCH   Log-Likelihood:               -3673.53
Distribution:                  Normal   AIC:                           7353.05
Method:            Maximum Likelihood   BIC:                           7370.98
                                        No. Observations:                 2913
Date:                Fri, Sep 27 2024   Df Residuals:                     2913
Time:                        18:53:28   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      2.4

In [207]:
rmse_egarch = np.sqrt(mse(realized_vol[-n:]/100,
                        np.sqrt(forecast_egarch.variance.iloc[-len(split_date):]/100
                                )
                        )
                    )

print('The RMSE value of EGARCH Model is {:.4f}'.format(rmse_egarch))


The RMSE value of EGARCH Model is 0.0904


In [208]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[1:], y=(realized_vol / 100), mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=forecast_garch.variance.index, y=(forecast_garch.variance['h.1']/100), mode='lines', name='Predict Volatility'))
fig.update_layout(
    title=f'Volatility Clustering of {stocks} - EGARCH',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

In [209]:
rmse = pd.DataFrame({
    'Model': ['ARCH', 'GARCH', 'GJR-GARCH', 'EGARCH'],
    'RMSE': [rmse_arch, rmse_garch, rmse_gjr_garch, rmse_egarch]
})

rmse

,Model,RMSE
0,ARCH,0.089637
1,GARCH,0.087802
2,GJR-GARCH,0.088007
3,EGARCH,0.090431


# Machine Learning Models

In [210]:
from sklearn.svm import SVR
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [211]:
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)

In [212]:
return_svm = ret ** 2
return_svm = return_svm.reset_index()
del return_svm['Date']

In [213]:
X = pd.concat([realized_vol, return_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X = X.drop('index', axis=1)

In [214]:

X

,0,1
0,0.133763,0.030512
1,0.537728,0.880063
2,0.658062,0.693210
3,0.646860,0.058875
4,0.823912,1.171401
...,...,...
2904,0.553360,0.056749
2905,0.586249,0.222071
2906,0.539227,0.000347
2907,0.548919,0.176917


In [215]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol = realized_vol.drop('index', axis=1)
realized_vol

,Adj Close
0,0.133763
1,0.537728
2,0.658062
3,0.646860
4,0.823912
...,...
2904,0.553360
2905,0.586249
2906,0.539227
2907,0.548919



Essas três linhas de código são exemplos de SVR (Support Vector Regression), uma versão do algoritmo de Support Vector Machine (SVM) usada para regressão. Vamos analisar cada parte do código para entender o que está acontecendo e como as diferentes opções de kernel influenciam o comportamento do modelo.

1. SVR (Support Vector Regression)
O SVR é uma extensão do SVM (Support Vector Machine), que originalmente foi desenvolvido para resolver problemas de classificação. No SVR, o objetivo é prever um valor contínuo (regressão), ao invés de classificar classes discretas. Ele funciona encontrando um hiperplano que melhor se ajusta aos dados, minimizando o erro de previsão dentro de uma margem de tolerância.

A ideia principal por trás do SVR é encontrar uma linha (ou hiperplano) que:

- Minimiza o erro de previsão dentro de uma margem tolerada (ϵ).
- Apenas os pontos fora dessa margem (ϵ) afetam o ajuste do modelo (os chamados vetores de suporte).


2. Kernel
O kernel no SVR (assim como no SVM) é uma função que define como os dados são transformados em um espaço de maior dimensão para que a regressão possa ser ajustada de forma mais eficaz. Ele permite que o SVR encontre padrões mais complexos nos dados.

- kernel='linear': O kernel linear busca um ajuste direto em uma linha reta (ou hiperplano no caso de dimensões maiores). Ele é usado quando os dados podem ser separados ou ajustados bem com uma linha reta.
 
- kernel='poly': O kernel polinomial permite que o SVR ajuste uma curva polinomial aos dados, em vez de uma linha reta. Isso é útil quando os dados não podem ser ajustados de forma precisa por uma linha reta e exigem uma relação mais complexa, como uma curva.

- kernel='rbf': O kernel radial basis function (RBF) é um dos mais populares. Ele permite ao SVR capturar padrões muito mais complexos, modelando relações não lineares. O RBF transforma os dados de tal forma que, no novo espaço, os pontos podem ser separados (ou ajustados) por uma linha reta, mesmo que no espaço original essa separação não seja possível.

In [216]:
svr_poly = SVR(kernel='poly', degree=2)
svr_linear = SVR(kernel='linear')
svr_rbf = SVR(kernel='rbf')

In [217]:
para_grid ={'gamma': sp_rand(),
            'C': sp_rand(),
            'epsilon': sp_rand()}

Os Parâmetros do SVR
- a) gamma
O parâmetro gamma controla o quanto uma única instância de treino afeta o ajuste do modelo no kernel RBF (ou polinomial). Um valor alto de gamma significa que cada ponto de dados tem um impacto maior no modelo, resultando em uma fronteira de decisão mais complexa. Um valor pequeno de gamma significa que os pontos de dados individuais têm menos impacto, o que leva a uma fronteira de decisão mais suave.
- b) C
O parâmetro C é o parâmetro de regularização. Ele controla o trade-off entre maximizar a margem da separação e minimizar o erro de classificação (no caso de SVM) ou a perda na regressão (no caso de SVR). Valores maiores de C tornam o modelo mais rigoroso em tentar ajustar todos os pontos de dados, enquanto valores menores de C permitem que o modelo seja mais flexível, ignorando alguns erros em troca de uma margem maior.
- c) epsilon
O parâmetro epsilon define uma zona ao redor da linha de regressão onde os erros são ignorados (o que chamamos de margem de tolerância). Ele controla o quão sensível o modelo SVR deve ser ao erro: dentro dessa margem, erros são "tolerados" e não são penalizados.

**3.** Função sp_rand()
A função **sp_rand()** não é uma função padrão de Python, então provavelmente faz referência a uma função customizada ou uma abreviação de alguma biblioteca como scipy.stats ou sklearn.model_selection, como randint() ou uniform(), que são usadas para gerar valores aleatórios.

- Se for baseada no scipy.stats, aqui estão algumas funções comuns que podem estar sendo usadas:

- scipy.stats.uniform(): Gera valores aleatórios de uma distribuição uniforme contínua dentro de um intervalo específico.
- scipy.stats.randint(): Gera números inteiros aleatórios de uma distribuição uniforme discreta dentro de um intervalo.
Essas funções são usadas para sorteios aleatórios durante a busca de hiperparâmetros, em vez de testar manualmente diferentes combinações de parâmetros.

4. Busca de Hiperparâmetros (Randomized Search)
A ideia por trás desse dicionário é realizar uma busca aleatória de hiperparâmetros, provavelmente com o método RandomizedSearchCV do Scikit-learn. Isso envolve:

- Sorteio de valores aleatórios de gamma, C e epsilon para testar várias combinações desses hiperparâmetros.
- Avaliar cada combinação de parâmetros com validação cruzada para determinar qual conjunto de parâmetros oferece o melhor desempenho para o seu modelo SVR.
- Como a busca é aleatória, ela pode encontrar boas combinações de parâmetros com menos tentativas do que uma busca exaustiva (como o GridSearchCV, que testa todas as combinações possíveis).

## Linear

In [219]:
clf = RandomizedSearchCV(svr_linear, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr = clf.predict(X.iloc[-n:])

In [220]:
predict_svr_lin = pd.DataFrame(predict_svr)
predict_svr_lin.index = ret.iloc[-n:].index

In [221]:
rmse_svr_lin = mean_squared_error(realized_vol.iloc[-n:], predict_svr_lin)
print('The RMSE value of SVR with Linear Kernel is: {:.6f}'.format(rmse_svr_lin))

The RMSE value of SVR with Linear Kernel is: 0.019924


In [222]:
realized_vol.index = ret.iloc[4:].index

In [157]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol['Adj Close'], mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=predict_svr_lin.index, y=predict_svr_lin[0], mode='lines', name='Predict Volatility SVR Linear'))
fig.update_layout(
    title='Volatility Clustering ',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()


## RBF



In [159]:
clf = RandomizedSearchCV(svr_rbf, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr = clf.predict(X.iloc[-n:])

In [160]:
predict_svr_rbf = pd.DataFrame(predict_svr)
predict_svr_rbf.index = ret.iloc[-n:].index

In [161]:
rmse_svr_rbf = mean_squared_error(realized_vol.iloc[-n:], predict_svr_rbf)
print('The RMSE value of SVR with RBF Kernel is: {:.6f}'.format(rmse_svr_rbf))

The RMSE value of SVR with RBF Kernel is: 0.001592


In [162]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol['Adj Close'], mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=predict_svr_rbf.index, y=predict_svr_rbf[0], mode='lines', name='Predict Volatility SVR RBF'))
fig.update_layout(
    title='Volatility Clustering ',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## Poly

In [164]:
clf = RandomizedSearchCV(svr_poly, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr = clf.predict(X.iloc[-n:])

In [165]:
predict_svr_poly = pd.DataFrame(predict_svr)
predict_svr_poly.index = ret.iloc[-n:].index

In [166]:
rmse_svr_poly = mean_squared_error(realized_vol.iloc[-n:], predict_svr_poly)
print('The RMSE value of SVR with Linear Kernel is: {:.6f}'.format(rmse_svr_poly))

The RMSE value of SVR with Linear Kernel is: 0.149459


In [167]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol['Adj Close'], mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=predict_svr_poly.index, y=predict_svr_poly[0], mode='lines', name='Predict Volatility SVR Polynomial'))
fig.update_layout(
    title='Volatility Clustering ',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

# Neural Network

## SK-learn

In [168]:
NN_vol = MLPRegressor(learning_rate_init=.001, random_state=1)
paragrid_NN = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
               'max_iter': [500, 1000],
               'alpha': [.00005, .0005]
               }

clf = RandomizedSearchCV(NN_vol, paragrid_NN)
clf.fit(X.iloc[:-n].values,
        realized_vol.iloc[1:-(n-1)].values.reshape(-1,)
        )
NN_predictions = clf.predict(X.iloc[-n:])


In [169]:
NN_predictions = pd.DataFrame(NN_predictions)
NN_predictions.index = ret.iloc[-n:].index

In [171]:
rmse_NN =  mean_squared_error(realized_vol.iloc[-n:], NN_predictions)
print('The RMSE value of NN is: {:.6f}'.format(rmse_NN))

The RMSE value of NN is: 0.004187


In [172]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol['Adj Close'], mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=NN_predictions.index, y=NN_predictions[0], mode='lines', name='Predict Volatility SVR Polynomial'))
fig.update_layout(
    title='Volatility Clustering ',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()

## Tensor Flow - Keras

In [173]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [174]:
model = keras.Sequential(
    [layers.Dense(256, activation='relu'),
     layers.Dense(128, activation='relu'),
     layers.Dense(1, activation='linear'),
     ]
)




In [175]:
model.compile(loss='mse', optimizer='rmsprop')

In [176]:
epochs_trial = np.arange(100, 400, 4)
batch_trial = np.arange(100, 400, 4)
DL_pred = []
DL_RMSE = []
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values,
              realized_vol.iloc[1:-(n-1)].values.reshape(-1,),
              batch_size=k, epochs=j, verbose=False)
    
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mean_squared_error(realized_vol[-n:], DL_predict.flatten())))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
DL_RMSE_1:0.079196
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
DL_RMSE_2:0.063286
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
DL_RMSE_3:0.069208
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
DL_RMSE_4:0.064408


In [177]:
if len(DL_pred) == len(DL_RMSE) and len(DL_RMSE) > 0:
    DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
    DL_predict.index = ret.iloc[-n:].index
else:
    print("As listas DL_pred e DL_RMSE têm tamanhos diferentes ou estão vazias.")


In [178]:
try:
    DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
    DL_predict.index = ret.iloc[-n:].index
except IndexError as e:
    print(f"Erro de índice: {e}")
    print(f"Tamanhos de DL_pred: {len(DL_pred)}, DL_RMSE: {len(DL_RMSE)}")


In [179]:
DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
DL_predict.index = ret.iloc[-n:].index

In [180]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=realized_vol.index, y=realized_vol['Adj Close'], mode='lines', name='Realized Volatility'))
fig.add_trace(go.Scatter(x=DL_predict.index, y=DL_predict[0], mode='lines', name='Predict Volatility TensorFlow - Keras'))
fig.update_layout(
    title='Volatility Clustering ',
    xaxis_title='Date',
    yaxis_title='Daily Returns',
    width=1000,
    height=600
)
fig.show()